Consideraciones iníciales:

Infelizmente no consigo escribir correctamente en español. Así que proseguiré con el desafío, escribiendo lo en Inglés.

# Data Scientist Challenge - LATAM Airlines

### Problem

The problem consists of predicting the delay probability of flights landing or taking off from Santiago de Chile (SCL). For this purpose, we provide a dataset using public and real data where each row corresponds to a flight that landed or took off from SCL. For each flight the following information is available:

- **Fecha-I** : Scheduled date and time of the flight.
- **Vlo-I** : Programmed flight number.
- **Ori-I** : Programmed origin city code.
- **Des-I** : Programmed destination city code.
- **Emp-I** : Programmed flight airline code.
- **Fecha-O** : Date and time of flight operation.
- **Vlo-O** : Flight operation flight number.
- **Ori-O** : Operation origin city code.
- **Des-O** : Operation destination city code.
- **Emp-O** : Operation flight airline code.
- **DIA** : Operation day of the month of the flight.
- **MES** : Flight operation month number.
- **AÑO** : Year of flight operation.
- **DIANOM** : Day of week of flight operation.
- **TIPOVUELO** : Flight type, I = International, N = National.
- **OPERA** : Operation airline name.
- **SIGLAORI** : Origin city name.
- **SIGLADES** : Destination city name.


### Challenge

1. How is the data distributed? What catches your attention or what is your conclusion about this?

2. Create the following additional columns then export them in a file `synthetic_features.csv`
    - **temporada_alta** : `1` if *Fecha-I* is between December 15th and March 3rd, or July 15th and July 31th, or September 11th and September 30, `0` otherwise
    - **dif_min** : time difference in minutes between *Fecha-O* and *Fecha-I*
    - **atraso_15** : `1` if *dif_min* > 15, `0` otherwise
    - **periodo_dia** : `mañana` (between 5:00 and 11:59), `tarde` (between 12:00 and 18:59) and `noche` (between 19:00 and 4:59), based on *Fecha-I*

3. How is delay rate composed by destination, airline, month, day of week, season (*temporada_alta*) and flight type? What features would you expect to better predict delay?

4. Train one or more models (using algorithms of your choice) to estimate the probability of flight delay. Feel free to create additional variables and/or add external variables.

5. Evaluate your model. How did it perform? Which metrics did you use to evaluate it and why? Why did you choose this algorithm in particular? Which features are more important to the prediction? How would you enhance the model performance?

In [1]:
import pandas as pd
from datetime import datetime

In [2]:
def translating_data(df):

    days_of_week = {
        'Lunes' : 'Monday',
        'Martes' : 'Tuesday',
        'Miercoles' : 'Wednesday',
        'Jueves' : 'Thursday',
        'Viernes' : 'Friday',
        'Sabado' : 'Saturday',
        'Domingo' : 'Sunday'
    }

    columns = [
        'datetime_i', 'flight_number_i', 'origin_code_i', 'destination_code_i', 'airline_code_i',
        'datetime_o', 'flight_number_o', 'origin_code_o', 'destination_code_o', 'airline_code_o',
        'day', 'month', 'year', 'week_day', 'flight_type', 'airline', 'origin', 'destination'
    ]

    df.columns = columns
    df.loc[:, 'week_day'] = df.loc[:, 'week_day'].apply(lambda x: days_of_week[x])

    return df

In [3]:
def get_treated_data():

    flight_data = pd.read_csv('data/dataset_SCL.csv', parse_dates=[0, 5], low_memory=False)
    flight_data = translating_data(flight_data)

    # It's necessary to create an index that is unique to identify each flight correctly
    flight_data['index'] = flight_data['datetime_o'].astype(str) + '_' + flight_data['flight_number_o'] + '_' +\
        flight_data['origin_code_o'] + '_' + flight_data['destination_code_o'] + '_' + flight_data['airline_code_o']
    flight_data = flight_data.set_index('index')

    return flight_data

In [10]:
def creating_synthetic_features(df):

    synth_df = df.copy()

    # Separating date and time
    synth_df['date_i'] = synth_df['datetime_i'].apply(lambda x: str(x.date()))
    synth_df['time_i'] = synth_df['datetime_i'].apply(lambda x: str(x.time()))

    synth_df['temporada_alta'] = synth_df['date_i'].apply(lambda x: 1 if x[5:] <= '03-03' or (x[5:] >= '07-15' and x[5:] <= '07-31') \
        or (x[5:] >= '09-11' and x[5:] <= '09-30') or x[5:] >= '12-15' else 0)

    synth_df['dif_min'] = synth_df['datetime_o'] - synth_df['datetime_i']
    synth_df.loc[:, 'dif_min'] = synth_df['dif_min'].apply(lambda x: x.total_seconds()/60).astype(int)

    synth_df['atraso_15'] = synth_df['dif_min'] > 15
    synth_df['atraso_15'] = synth_df['atraso_15'].astype(int)

    synth_df['periodo_dia'] = synth_df['time_i'].apply(lambda x: 'mañana' if x >= '05:00:00' and x < '12:00:00' \
        else 'tarde' if x >= '12:00:00' and x < '19:00:00' else 'noche')

    return synth_df[['temporada_alta', 'dif_min', 'atraso_15', 'periodo_dia']]

In [5]:
def saving_synthetic_features(synth_df):

    synth_df.to_csv('data/synthetic_features.csv')

In [11]:
df = get_treated_data()
synth_df = creating_synthetic_features(df)
saving_synthetic_features(synth_df)

df = df.merge(right=synth_df, left_index=True, right_index=True)
df.head()

,datetime_i,flight_number_i,origin_code_i,destination_code_i,airline_code_i,datetime_o,flight_number_o,origin_code_o,destination_code_o,airline_code_o,...,year,week_day,flight_type,airline,origin,destination,temporada_alta,dif_min,atraso_15,periodo_dia
index,,,,,,,,,,,,,,,,,,,,,
2017-01-01 23:33:00_226_SCEL_KMIA_AAL,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,...,2017,Sunday,I,American Airlines,Santiago,Miami,1,3,0,noche
2017-01-02 23:39:00_226_SCEL_KMIA_AAL,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,...,2017,Monday,I,American Airlines,Santiago,Miami,1,9,0,noche
2017-01-03 23:39:00_226_SCEL_KMIA_AAL,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,...,2017,Tuesday,I,American Airlines,Santiago,Miami,1,9,0,noche
2017-01-04 23:33:00_226_SCEL_KMIA_AAL,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,...,2017,Wednesday,I,American Airlines,Santiago,Miami,1,3,0,noche
2017-01-05 23:28:00_226_SCEL_KMIA_AAL,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,...,2017,Thursday,I,American Airlines,Santiago,Miami,1,-2,0,noche


### Analysing the data

In [16]:
df[['flight_number_i', 'origin_code_i', 'destination_code_i', 'airline']].value_counts()

flight_number_i  origin_code_i  destination_code_i  airline                 
7                SCEL           SCCI                Sky Airline                 365
285              SCEL           SCTE                Grupo LATAM                 364
190              SCEL           SCAR                Sky Airline                 364
273              SCEL           SCTE                Grupo LATAM                 364
802              SCEL           SPJC                Sky Airline                 364
                                                                               ... 
503              SCEL           LEMD                Plus Ultra Lineas Aereas      1
51               SCEL           SCQP                Grupo LATAM                   1
1186             SCEL           SCAR                Grupo LATAM                   1
1178             SCEL           SCAR                Grupo LATAM                   1
9956             SCEL           SPJC                Grupo LATAM                   1